# **Bitcoin price prediction - Generalized Linear Regression**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: executing the chosen model, first with default values, then by choosing the best parameters by performing hyperparameter tuning with cross validation and performance evaluation. Finally validate the tuned model and train it on the whole train / validation set

# Global constants, dependencies, libraries and tools

In [ ]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = True # Decide whether or not to use operations that might slow down notebook execution
MODEL_NAME = "GeneralizedLinearRegression"
MAIN_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [ ]:
if not LOCAL_RUNNING: 
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(MAIN_DIR, force_remount=True)

In [ ]:
# Set main dir
MAIN_DIR = MAIN_DIR + "" if LOCAL_RUNNING else "/MyDrive/BDC/project"

###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets names
DATASET_TRAIN_VALID_NAME = "bitcoin_blockchain_data_15min_train_valid"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features labels
FEATURES_LABEL = "features"
TARGET_LABEL = "next-market-price"

# Features names
ALL_FEATURES_NAME = "all_features"
MOST_CORR_FEATURES_NAME = "most_corr_features"
LEAST_CORR_FEATURES_NAME = "least_corr_features"

# Features paths
ALL_FEATURES = FEATURES_DIR + "/" + ALL_FEATURES_NAME + ".json"
MOST_CORR_FEATURES = FEATURES_DIR + "/" + MOST_CORR_FEATURES_NAME + ".json"
LEAST_CORR_FEATURES = FEATURES_DIR + "/" + LEAST_CORR_FEATURES_NAME + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

#####################
# --- UTILITIES --- #
#####################

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/short_term_split"

# Results path
MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + ".csv"
MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.json"

In [ ]:
# Suppression of warnings for better reading
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
if not LOCAL_RUNNING:
    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

# Import files

In [ ]:
# Import my files
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import utilities, parameters

importlib.reload(utilities)
importlib.reload(parameters)

# Create the pyspark session

In [ ]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [ ]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [ ]:
def dataset_info(dataset):
  # Print dataset
  dataset.show(3)

  # Get the number of rows
  num_rows = dataset.count()

  # Get the number of columns
  num_columns = len(dataset.columns)

  # Print the shape of the dataset
  print("Shape:", (num_rows, num_columns))

  # Print the schema of the dataset
  dataset.printSchema()

In [ ]:
if SLOW_OPERATIONS:
  dataset_info(df)

# Loading features

In [ ]:
# Loading all the features
with open(ALL_FEATURES, "r") as f:
    ALL_FEATURES = json.load(f)
print(ALL_FEATURES)

In [ ]:
# Loading the most correlated features
with open(MOST_CORR_FEATURES, "r") as f:
    MOST_CORR_FEATURES = json.load(f)
print(MOST_CORR_FEATURES)

In [ ]:
# Loading least correlated features
with open(LEAST_CORR_FEATURES, "r") as f:
    LEAST_CORR_FEATURES = json.load(f)
print(LEAST_CORR_FEATURES)

# ❗Model train / validation
In order to train and validate the model, I'll try several approaches:
- **Default without normalization:** Make predictions using the chosen base model
- **Default with normalization:** Like the previous one but features are normalized

At this point, the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:

- **Hyperparameter tuning:** Researching the best parameters to use
- **Cross Validation:** validate the performance of the model with the chosen parameters
---
For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem):

**Block time series split**

Involves dividing the time series into blocks of equal length, and then using each block as a separate fold for cross-validation.

❗TODO: INSERT IMAGE HERE

**Walk forward time series split**

Involves using a sliding window approach to create the training and validation sets for each fold. The model is trained on a fixed window of historical data, and then tested on the next observation in the time series. This process is repeated for each subsequent observation, with the window sliding forward one step at a time. 

❗TODO: INSERT IMAGE HERE

**Short term time series split**

Involves dividing the time series considering as validation set a narrow period of time (3 months in our case) and as train set everything that happened before this period, in such a way as to best benefit from the trend in the short term.

❗TODO: INSERT IMAGE HERE

In [ ]:
# Get splitting parameters based on the choosen splitting method
splitting_info = parameters.get_splitting_params('short_term_split')
splitting_info

## Default
The train / validation set will be splitted Based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

### Without normalization

In [ ]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [ ]:
# Get default parameters
params = parameters.get_defaults_model_params(MODEL_NAME)
params

In [ ]:
# Make predictions by using all the features
default_res_all, default_pred_all = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_all

In [ ]:
utilities.show_results(default_pred_all, MODEL_NAME)

In [ ]:
# Make predictions by using the most correlated features
default_res_most_corr, default_pred_most_corr = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_CORR_FEATURES, MOST_CORR_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_most_corr

In [ ]:
utilities.show_results(default_pred_most_corr, MODEL_NAME)

In [ ]:
# Make predictions by using the least correlated features
default_res_least_corr, default_pred_least_corr = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_CORR_FEATURES, LEAST_CORR_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_least_corr

In [ ]:
utilities.show_results(default_pred_least_corr, MODEL_NAME)

### With normalization

In [ ]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [ ]:
# Valid performances with all the features
default_norm_res_all, default_norm_pred_all = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_all

In [ ]:
utilities.show_results(default_norm_pred_all, MODEL_NAME)

In [ ]:
# Make predictions by using the most the features
default_norm_res_most_corr, default_norm_pred_most_corr = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_CORR_FEATURES, MOST_CORR_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_most_corr

In [ ]:
utilities.show_results(default_norm_pred_most_corr, MODEL_NAME)

In [ ]:
# Make predictions by using the least the features
default_norm_res_least_corr, default_norm_pred_least_corr = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_CORR_FEATURES, LEAST_CORR_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_least_corr

In [ ]:
utilities.show_results(default_norm_pred_least_corr, MODEL_NAME)

In [ ]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [ ]:
# Define the results to show
default_comparison_lst = [default_res_all, default_res_most_corr, default_res_least_corr, default_norm_res_all, default_norm_res_most_corr, default_norm_res_least_corr]

# Show the comparison table
default_comparison_lst_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

❗TODO: INSERT DEFAULT RECAP HERE

## ❗Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration. 

For each split, all combinations of the model parameters are tested and those that return a lower RMSE are considered. Using the previously selected parameters, cross valdiation is performed to see overall performance.

### ❗Hyperparameter tuning

In [ ]:
# From now on, only selected and normalized features will be considered
MODEL_TYPE = "hyp_tuning"
CHOSEN_FEATURES = MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = MOST_CORR_FEATURES_NAME
FEATURES_NORMALIZATION = True

In [ ]:
# Get model grid parameters
params = parameters.get_model_grid_params(MODEL_NAME)
params

In [ ]:
# Perform hyperparameter tuning
hyp_res = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
hyp_res

❗TODO: INSERT HYP TUNING RECAP HERE

---
To select the best parameters to be used in the final model I assign a score to each value in the "Parameters" column based on the following criteria:
* Calculate the frequency of each unique value in the "Parameters" column.
* Normalize the frequencies to a scale of 0 to 1, where 1 represents the highest frequency.
* Calculate the split weight for each value in the "Parameters" column, where a higher split number corresponds to a higher weight.
* Normalize the split weights to a scale of 0 to 1, where 1 represents the highest split weight.
* Calculate the RMSE weight for each value in the "Parameters" column, where a lower RMSE value corresponds to a higher weight.
* Normalize the RMSE weights to a scale of 0 to 1, where 1 represents the highest RMSE weight.
* Calculate the overall score for each value in the "Parameters" column by combining the normalized frequency, split weight, and RMSE weight.

I take into consideration the parameters that have the highest score.

In [ ]:
grouped_scores, best_params = parameters.choose_best_params(hyp_res)
grouped_scores

### ❗Cross validation

In [ ]:
MODEL_TYPE = "cross_val"

In [ ]:
# Get tuned parameters
params = parameters.get_best_model_params(best_params, MODEL_NAME)
params

In [ ]:
# Perform cross validation
cv_res, cv_pred = utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
cv_res

In [ ]:
utilities.show_results(cv_pred, MODEL_NAME)

In [ ]:
# Define the results to show
tuned_comparison_lst = [cv_res]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

# Train final model
At this point it is possible to tow the final model over the entire train/validation set in order to use it in the final step, that is, to make predictions about the data contained in the test set.

In [ ]:
MODEL_TYPE = "final_trained"

In [ ]:
# Train the final model on the whole train / validation set
train_res, final_model, train_pred = utilities.evaluate_trained_model(df, params, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
train_res

In [ ]:
utilities.show_results(train_pred, MODEL_NAME)

# Comparison table
Visualization of model performance at various stages of train / validation

In [ ]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df, train_res], ignore_index=True))
final_comparison_lst_df

# ❗Model accuracy

❗TODO: INSERT FORMULA AND EXPLANATION HERE

In [ ]:
# Convert the pandas dataset to a PySpark dataset
final_valid_pred_spark = spark.createDataFrame(cv_pred)

# Compute model accuracy
accuracy = utilities.model_accuracy(final_valid_pred_spark)
accuracy = f"Percentage of correct predictions for {MODEL_NAME, MODEL_TYPE}: {accuracy:.2f}%"

print(accuracy)

❗TODO: INSERT FINAL CONSIDERATION HERE

# Saving final results


In [ ]:
# Saving final model
final_model.write().overwrite().save(MODEL)

In [ ]:
# Saving final model results
final_comparison_lst_df.to_csv(MODEL_RESULTS, index=False)

In [ ]:
# Saving final accuracy results
with open(MODEL_ACCURACY_RESULTS, 'w') as file:
    json.dump(accuracy, file)